In [ ]:
!pip install transformers -U
!pip install accelerate -U

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='new_processed_recipes_data.txt',
    block_size=256
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

training_args = TrainingArguments(
    output_dir='./output',
    overwrite_output_dir=True,
    num_train_epochs=30,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()
trainer.save_model("./gpt2-fine-tuned")
tokenizer.save_pretrained('./gpt2-tokenizer')

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('./gpt2-fine-tuned')
tokenizer = GPT2Tokenizer.from_pretrained('./gpt2-tokenizer')

prompt = "[Q] Ingredients: potatoes, butter, milk, salt, pepper"

input_ids = tokenizer.encode(prompt, return_tensors='pt')
output = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id, top_k=70, top_p=0.9)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

In [ ]:
!zip -r /content/gpt2-fine-tuned.zip ./gpt2-fine-tuned
!zip -r /content/gpt2-tokenizer.zip ./gpt2-tokenizer

In [ ]:
from google.colab import files
files.download("./gpt2-fine-tuned.zip")
files.download("./gpt2-tokenizer.zip")